In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Activation, Layer, Dense
from packages.Utility.CNN.Callbacks import SaveModel, ValidateModel, StopTraining
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, GaborParams, RotatedGaborParams

Using TensorFlow backend.


In [2]:
fold_index = 0
dataset_dir = 'dataset/bin/DG321'
trained_dir = 'trained/DG321 LN[1]'

# Memuat Dataset

In [3]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Pra-Training

In [4]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model)

# Mendefinisikan Model CNN
Mendefinisikan arsitektur CNN

In [5]:
model = Sequential()

c1 = Conv2D(name='c1', filters = 6,
            kernel_size = 5, strides = 1, padding = 'valid', activation = 'relu', 
            input_shape = (32, 32, 1))
mp1 = MaxPool2D(pool_size = 2, name = 'mp1')

c2 = Conv2D(name='c2', filters = 16, 
            kernel_size = 5, strides = 1, padding = 'valid', activation = 'relu')
mp2 = MaxPool2D(pool_size = 2, name = 'mp2')

# Layer 1
model.add(c1)
model.add(mp1)

# Layer 2
model.add(c2)
model.add(mp2)

model.add(Flatten(name = 'flatten'))

# Fully Connected
model.add(Dense(120, activation = 'sigmoid', name = 'd1'))
model.add(Dense(84, activation = 'sigmoid', name = 'd2'))
model.add(Dense(10, activation = 'softmax', name = 'output'))

# Melakukan Pelatihan
Melakukan pelatihan pada CNN

In [6]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = model.fit(X_train, Y_train_onehot, epochs = 1, verbose = 1, batch_size = 250, callbacks=[val_model, save_model, stop_train])

Epoch 1/1
37500/37500 [==============================] - 41s 1ms/step
val_acc : 0.9480266666475932
Model Saved
New best accuracy : 0.9480266666475932
